In [2]:
import torch
from GeospatialFM.models.low_rank_attention import LowRankBlock
from timm.models.vision_transformer import Block
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
lrb = LowRankBlock(768, 12, qkv_bias=True, seperate_v=True).to('cuda:3')
block = Block(768, 12, qkv_bias=True).to('cuda:3')

In [12]:
lrb.attn.nqkv

6

In [13]:
x = torch.randn(64, 196, 13, 768).to('cuda:3')
x2 = torch.randn(64, 196*13, 768).to('cuda:3')

In [14]:
with torch.no_grad():
    o = lrb(x)
o.shape

torch.Size([64, 196, 13, 768])

In [14]:
# print the number of parameters
print('LowRankBlock:', sum(p.numel() for p in lrb.parameters()))
print('Block:', sum(p.numel() for p in block.parameters()))

LowRankBlock: 8859648
Block: 7087872


In [6]:
o.shape

torch.Size([64, 196, 13, 768])

In [25]:
# for every sub_module, print the number of parameters
for (name, param_lrb), (_, param_b) in zip(lrb.named_parameters(), block.named_parameters()):
    print(name, '\t', param_lrb.numel(), '\t', param_b.numel())

norm1.weight 	 768 	 768
norm1.bias 	 768 	 768
attn.qkv.weight 	 3538944 	 1769472
attn.qkv.bias 	 4608 	 2304
attn.proj.weight 	 589824 	 589824
attn.proj.bias 	 768 	 768
norm2.weight 	 768 	 768
norm2.bias 	 768 	 768
mlp.fc1.weight 	 2359296 	 2359296
mlp.fc1.bias 	 3072 	 3072
mlp.fc2.weight 	 2359296 	 2359296
mlp.fc2.bias 	 768 	 768


In [10]:
with torch.no_grad():
    o = block(x2)
o.shape

torch.Size([64, 2548, 768])

In [3]:
qkv = torch.nn.Linear(768, 768*6, bias=True).to('cuda:3')
x = torch.randn(64, 196, 13, 768).to('cuda:3')
B, N, C, D = x.shape

In [4]:
o = qkv(x).reshape(B, N, C, 6, 12, 64).permute(3, 0, 4, 1, 2, 5)

In [5]:
o.shape

torch.Size([6, 64, 12, 196, 13, 64])

In [20]:
qc, kc, qs, ks, vc, vs = o.unbind(0)
# qc, kc, qs, ks = self.q_norm(qc), self.k_norm(kc), self.q_norm(qs), self.k_norm(ks)
qs, ks, vs = qs.transpose(-2, -3), ks.transpose(-2, -3), vs.transpose(-2, -3)

In [21]:
qc.shape, kc.shape, qs.shape, ks.shape, vc.shape, vs.shape

(torch.Size([64, 12, 196, 13, 64]),
 torch.Size([64, 12, 196, 13, 64]),
 torch.Size([64, 12, 13, 196, 64]),
 torch.Size([64, 12, 13, 196, 64]),
 torch.Size([64, 12, 196, 13, 64]),
 torch.Size([64, 12, 13, 196, 64]))

In [22]:
xs = F.scaled_dot_product_attention(
                qs, ks, vs,
                dropout_p=0,
            ).transpose(-2, -3)

In [23]:
xs.shape


torch.Size([64, 12, 196, 13, 64])

In [24]:
xc = F.scaled_dot_product_attention(
                qc, kc, vc,
                dropout_p=0,
            )

In [25]:
xc.shape

torch.Size([64, 12, 196, 13, 64])

In [26]:
x = xs * xc
x.shape

torch.Size([64, 12, 196, 13, 64])